In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import timeit

os.environ["DGLBACKEND"] = "pytorch"

from dgl import from_networkx
import networkx as nx


import torch as th
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
import itertools

from src.models import EGRAPHSAGE, EGAT, EGCN
from src.dataset.dataset_info import datasets
from src.graph.graph_measures import calculate_graph_measures

# datasets = {dataset.name: dataset for dataset in datasets_list}

num_epochs = 5
batch_size = 128
learning_rate = 0.001
LAMBD_1 = 0.0001
LAMBD_2 = 0.001

In [2]:
name = "cic_ids_2017_5_percent"
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017"
# name = "nf_bot_iot"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"

dataset = datasets[name]

In [3]:
with open("datasets/" + name + "/training_graph.pkl", "rb") as f:
    G = pickle.load(f)

In [4]:
G = from_networkx(G,edge_attrs=['h',dataset.label_col] )

In [5]:
num_features = G.edata['h'].shape[1]
num_features

32

In [6]:
G.ndata['h'] = th.ones(G.num_nodes(), num_features)

G.ndata['h'] = th.reshape(G.ndata['h'], (G.ndata['h'].shape[0], 1, num_features))
G.edata['h'] = th.reshape(G.edata['h'], (G.edata['h'].shape[0], 1, num_features))

G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype=th.bool)

In [7]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(G.edata[dataset.label_col].cpu().numpy()),
                                                 y = G.edata[dataset.label_col].cpu().numpy())

In [8]:
class_weights = th.FloatTensor(class_weights)

criterion = nn.CrossEntropyLoss(weight = class_weights)

In [9]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [10]:
def train_gnn(model_class, Graph, residual, num_neighbors = None):
    node_features = Graph.ndata['h']
    edge_features = Graph.edata['h']

    edge_label = Graph.edata[dataset.label_col]
    train_mask = Graph.edata['train_mask']

    # model = EGRAPHSAGE(num_features, num_features, 128, F.relu,
    #                    dropout=0.2, num_neighbors=4, residual=residual)

    if num_neighbors:
        model = model_class(num_features, num_features, 128, F.relu,
                    dropout=0.2, num_neighbors=num_neighbors, residual=residual)
    else:
        model = model_class(num_features, num_features, 128, F.relu,
                    dropout=0.2, residual=residual)

    opt = th.optim.Adam(model.parameters())

    for epoch in range(1,num_epochs):
    #     pred = model(G, node_features,edge_features).cuda()
        pred = model(Graph, node_features,edge_features)
        loss = criterion(pred[train_mask] ,edge_label[train_mask])
        opt.zero_grad()
        loss.backward()
        opt.step()
        print('Epoch:', epoch ,' Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))

    return model

### Training EGCN

In [11]:
model_gcn = train_gnn(EGCN, G, False, None)

Epoch: 1  Training acc: 0.1989421248435974
Epoch: 2  Training acc: 0.2026335448026657
Epoch: 3  Training acc: 0.39573007822036743
Epoch: 4  Training acc: 0.9731689691543579


### Training EGCN residual

In [12]:
model_gcn_res = train_gnn(EGCN, G, True, None)

Epoch: 1  Training acc: 0.340955525636673
Epoch: 2  Training acc: 0.6718987226486206
Epoch: 3  Training acc: 0.7870001196861267
Epoch: 4  Training acc: 0.8211836814880371


### Training EGraphSage 

In [26]:
model_sage = train_gnn(EGRAPHSAGE, G, False, 4)

Epoch: 1  Training acc: 0.29976335167884827
Epoch: 2  Training acc: 0.7170047760009766
Epoch: 3  Training acc: 0.7779384851455688
Epoch: 4  Training acc: 0.8358482122421265


### Training EGraphSage residual

In [ ]:
model_sage_res = train_gnn(EGRAPHSAGE, G, True, 4)

### Training EGAT

In [39]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn


class GATLayer(nn.Module):
    # in_dim = 55
    # edim = 55
    # out_dim = z = 55 * 2

    def __init__(self, ndim_in, edim, ndim_out, activation):
        super(GATLayer, self).__init__()
        # equation (1)
#        print(edim)
        self.linear = nn.Linear(ndim_in + edim, ndim_out, bias=False)
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        # equation (2)

        self.attn_fc = nn.Linear(2*ndim_in, 1)
        self.reset_parameters()
        # print("OK1")

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        # print(edges.src["h"].shape)
        # print(edges.dst["h"].shape)
        # print(edges.data["h"].shape[2])
        # print("OKedge")
        z2 = th.cat([edges.src["h"], edges.dst["h"]], dim=2)
        # print("OKedge1")
        # print(z2.shape)
        a = self.attn_fc(z2)
        # print("OKedge2")
        # print(f'attention={F.leaky_relu(a)}')
        return {"e": F.leaky_relu(a)}

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.linear.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        # print("OK2")
        # print("DONE8")
        # b=th.cat([edges.src['h'], edges.data['h']], 1)
        # print(b.shape)
        # print(edges.src['h'].shape)
        # print(edges.data['h'].shape)
        return {"m": self.linear(th.cat([edges.src['h'], edges.data['h']], 2)), "e": edges.data["e"]}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        # print("DONE7")

        alpha = F.softmax(nodes.mailbox['e'], dim=1)
 #       print({nodes.mailbox['e']})
        # print(f'alpha={alpha.shape}')
       # print("DONE7")
        # equation (4)
#         z = th.mean(alpha * nodes.mailbox['m'],dim=1)
        z = th.sum(alpha * nodes.mailbox['m'], dim=1)
        # print(f'z.shape={z.shape}')
        # z = th.reshape(z, (z.shape[0], 1,z.shape[2]))
        # print("DONE9")
        # print

        return {'z': z}

        # print("DONE6")

    def forward(self, g, nfeats, efeats):
        with g.local_scope():
            g = g
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
          #  print("OK1")
            # equation (2)

            g.apply_edges(self.edge_attention)
         #   print("OK2")
            # equation (3) & (4)
            # self.g.update_all(self.message_func, self.reduce_func)
            g.update_all(self.message_func, self.reduce_func)

  #          print("OK3")
   #         print(g.ndata["z"])
    #        print(g.ndata["z"].shape)
     #       print(g.ndata["h"].shape)

            g.ndata['h'] = F.relu(self.W_apply(
                th.cat([g.ndata['h'], g.ndata['z']], 2)))
#            print(g.ndata['h'].shape)

            # g.ndata['h'] = th.reshape( g.ndata['h'], ( g.ndata['h'].shape[0], 1, g.ndata['h'].shape[2]))
            # g.ndata['h'] = g.ndata['h'][:, 0:1, :]
            # print(g.ndata['h'])
 #           print("OK4")
            return g.ndata['h']


class GAT(nn.Module):
    def __init__(self, ndim_in, edim, ndim_out, activation, dropout):
        super().__init__()
       # self.layers = nn.ModuleList([
        self.layers = nn.ModuleList()
        self.layers.append(GATLayer(ndim_in, edim, 55, activation))

        self.layers.append(GATLayer(55, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

       # print("DONE")

    def forward(self, g, nfeats, efeats):
        for i, layer in enumerate(self.layers):
            nfeats = layer(g, nfeats, efeats)

        # print('hello')
#         print(f"nfeats.shape: {nfeats.shape}")
#         print(f"nfeats.sum(1).shape: {nfeats.sum(1).shape}")
#         print("===================")
        return nfeats.sum(1)


class MLPPredictor(nn.Module):
    def __init__(self, in_features, edim, out_classes, residual):
        super().__init__()
        self.residual = residual
        if residual:
            self.W = nn.Linear(in_features * 2 + edim, out_classes)
        else:
            self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']

        h_v = edges.dst['h']
        if self.residual:
            h_uv = edges.data['h']
            h_uv = h_uv.view(h_uv.shape[0], h_uv.shape[2])
            score = self.W(th.cat([h_u, h_v, h_uv], 1))
        else:
            score = self.W(th.cat([h_u, h_v], 1))

        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']


class EGAT(nn.Module):
    def __init__(self, ndim_in, edim, ndim_out, activation, dropout, residual):
        super().__init__()
        self.gnn = GAT(ndim_in, edim, ndim_out, activation, dropout)

        self.pred = MLPPredictor(ndim_out, edim, 2, residual)
        # print("DONE")

    def forward(self, g, nfeats, efeats):
        h = self.gnn(g, nfeats, efeats)
        # print("DONE")
        return self.pred(g, h)


In [40]:
model_gat = train_gnn(EGAT, G, False, None)

Epoch: 1  Training acc: 0.3962155282497406
Epoch: 2  Training acc: 0.9441129565238953
Epoch: 3  Training acc: 0.9864479303359985


### Training EGAT residual

In [ ]:
model_gat_res = train_gnn(EGAT, G, True, None)

# Testing

In [ ]:
with open("datasets/" + name + "/testing_graph.pkl", "rb") as f:
    G_test = pickle.load(f)

In [ ]:
G_test = from_networkx(G_test,edge_attrs=['h',dataset.label_col] )

In [ ]:
actual_label = G_test.edata.pop(dataset.label_col)
G_test.ndata['feature'] = th.ones(G_test.num_nodes(),num_features)

In [ ]:
G_test.ndata['feature'] = th.reshape(G_test.ndata['feature'], (G_test.ndata['feature'].shape[0], 1, G_test.ndata['feature'].shape[1]))
G_test.edata['h'] = th.reshape(G_test.edata['h'], (G_test.edata['h'].shape[0], 1, G_test.edata['h'].shape[1]))

In [ ]:
def test_gnn(model, graph):
    start_time = timeit.default_timer()
    node_features_test = graph.ndata['feature']
    edge_features_test = graph.edata['h']
    # test_pred = model(G_test, node_features_test, edge_features_test).cuda()
    test_pred = model(graph, node_features_test, edge_features_test)
    elapsed = timeit.default_timer() - start_time
    print(str(elapsed) + ' seconds')

    test_pred = test_pred.argmax(1)
    test_pred = th.Tensor.cpu(test_pred).detach().numpy()

    actual = ["Normal" if i == 0 else "Attack" for i in actual_label]
    test_pred = ["Normal" if i == 0 else "Attack" for i in test_pred]

    return (actual, test_pred)


In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

### Testing EGCN

In [ ]:
actual, test_pred = test_gnn(model_gcn, G_test)

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))

### Testing EGCN residual

In [ ]:
actual, test_pred = test_gnn(model_gcn_res, G_test)

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))

### Testing EGraphSage

In [ ]:
actual, test_pred = test_gnn(model_sage, G_test)

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))

### Testing EGraphSage residual

In [ ]:
actual, test_pred = test_gnn(model_sage_res, G_test)

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))

### Testing EGAT

In [ ]:
actual, test_pred = test_gnn(model_gat, G_test)

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))

### Testing EGAT residual

In [ ]:
actual, test_pred = test_gnn(model_gat_res, G_test)

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))